# NovaStreamParser Tutorial

This notebook provides a step-by-step guide to using the NovaStreamParser package for parsing and processing streaming responses from AWS Bedrock Nova models.

## Overview

NovaStreamParser provides decorators that enable extraction of content between specified XML tags from both `invoke_model_with_response_stream` and `converse_stream` API responses. This is particularly useful for extracting reasoning content from `<thinking>` tags or other structured XML content in model responses.

## Step 1: Import Required Libraries

First, let's import all the necessary libraries and modules:

In [ ]:
import boto3
import json
from NovaStreamParser.nova_parsed_event_stream import (
    parse_invoke_model_with_response_stream,
    parse_converse_stream
)

print("✅ Libraries imported successfully")

## Step 2: Define Decorated Stream Processing Functions

The NovaStreamParser uses decorators to wrap your stream processing functions. Let's create two decorated functions - one for each API type:

In [ ]:
# Decorator for converse_stream API
@parse_converse_stream("thinking")
def process_converse_stream(response_stream):
    """Process converse_stream responses and extract content from <thinking> tags"""
    return response_stream

# Decorator for invoke_model_with_response_stream API
@parse_invoke_model_with_response_stream(target_tag_name="thinking")
def process_invoke_model_with_response_stream(response_stream):
    """Process invoke_model_with_response_stream responses and extract content from <thinking> tags"""
    return response_stream

print("✅ Decorated functions defined")
print("   - process_converse_stream: Extracts content from <thinking> tags in converse_stream")
print("   - process_invoke_model_with_response_stream: Extracts content from <thinking> tags in invoke_model_with_response_stream")

## Step 3: Configure the System Prompt and Instructions

Let's set up the system prompt that instructs the model to use `<thinking>` tags for reasoning:

In [ ]:
# Base system instructions
system_text = """
You are a friend and helpful assistant that answers questions about the weather. 
The user and you will engage in a dialog exchanging the transcripts of a natural real-time conversation. Keep your responses short, generally two or three sentences for chatty scenarios.
You can use the tool to get the weather for a city.
You can use the tool multiple times to get the weather for multiple cities.
You can use the tool to get the weather for multiple cities at once.
"""

# Important: Instruct the model to use <thinking> tags
postamble = "\nWhen reasoning on your replies, place the reasoning in <thinking></thinking> tags."

# Combine into system prompt format
system_prompt = [{"text": system_text + postamble}]

print("✅ System prompt configured")
print(f"   - Base instructions: {len(system_text)} characters")
print(f"   - Includes instruction to use <thinking> tags for reasoning")

## Step 4: Set Up Inference Configuration

Configure the model's inference parameters:

In [ ]:
# Inference configuration
inference_config = {
    "maxTokens": 1024,    # Maximum tokens to generate
    "topP": 0.9,          # Nucleus sampling parameter
    "temperature": 0.7    # Controls randomness (0.0 = deterministic, 1.0 = very random)
}

print("✅ Inference configuration set:")
for key, value in inference_config.items():
    print(f"   - {key}: {value}")

## Step 5: Define Tool Configuration

Set up the weather tool that the model can use:

In [ ]:
# Tool configuration for weather queries
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "getWeather",
                "description": "A tool to get the weather",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "city": {
                                "type": "string",
                                "description": "A tool to get the weather for a particular city."
                            }
                        },
                        "required": ["city"]
                    }
                }
            }
        }
    ],
    "toolChoice": {
        "auto": {}  # Let the model decide when to use tools
    }
}

print("✅ Tool configuration defined:")
print(f"   - Tool name: {tool_config['tools'][0]['toolSpec']['name']}")
print(f"   - Tool choice: Auto (model decides when to use)")

## Step 6: Create Conversation Messages

Set up a realistic conversation flow that demonstrates tool usage and thinking:

In [ ]:
# Conversation messages demonstrating a natural flow
messages = [
    {
        "role": "user",
        "content": [
            {
                "text": "Hi what's the weather?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "Hi there! Could you please tell me which city you're interested in?"
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "text": "Yes, I'm in Seattle, Washington"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "<thinking>I need to get the weather for Seattle, Washington.</thinking>"
            },
            {
                "toolUse": {
                    'name': 'getWeather',
                    'toolUseId': '4356828f-a39c-4e4e-b9d5-dcf6027a4c7a',
                    'input': {
                        'city': 'Seattle, Washington'
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "text": "Actually, I'm in Tacoma, Washington."
            }
        ]
    }
]

print("✅ Conversation messages created:")
print(f"   - Total messages: {len(messages)}")
print(f"   - Includes example of <thinking> tags in assistant response")
print(f"   - Demonstrates tool usage scenario")

## Step 7: Prepare Request Body and Model Configuration

Combine all configurations into the request body:

In [ ]:
# Combine all configurations
body = {
    "system": system_prompt,
    "inferenceConfig": inference_config,
    "toolConfig": tool_config,
    "messages": messages
}

# Model configuration
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
modelId = LITE_MODEL_ID

print("✅ Request body prepared:")
print(f"   - Model ID: {modelId}")
print(f"   - Body contains: {list(body.keys())}")

## Step 8: Initialize AWS Bedrock Client

Create the Bedrock Runtime client:

In [ ]:
# Create a Bedrock Runtime client in the AWS Region of your choice
client = boto3.client("bedrock-runtime", region_name="us-east-1")

print("✅ AWS Bedrock Runtime client created")
print(f"   - Region: us-east-1")
print(f"   - Service: bedrock-runtime")

# Verify client configuration
try:
    # This will help verify AWS credentials are configured
    print(f"   - Client region: {client.meta.region_name}")
except Exception as e:
    print(f"   - Warning: Could not verify client configuration: {e}")

Here is a little helper function to just print out the events of the stream.

In [ ]:
def print_helper(event, mode = ""):
    block = None
    
    if (mode == "CONVERSE"):
        block = event
    else:
        chunk = event.get("chunk")
        block = json.loads(chunk.get("bytes").decode())

    if (block is not None
        and block.get("contentBlockDelta") is not None
        and block["contentBlockDelta"].get("delta") is not None
        and block["contentBlockDelta"]["delta"].get("text") is not None):
        print(f"{block["contentBlockDelta"]["delta"]["text"]}", end="")
    elif (block is not None
        and block.get("contentBlockDelta") is not None
        and block["contentBlockDelta"].get("delta") is not None
        and block["contentBlockDelta"]["delta"].get("toolUse") is not None):
        print(f"{block["contentBlockDelta"]["delta"]["toolUse"]}", end="")
    elif (block is not None
        and block.get("contentBlockStart") is not None
        and block["contentBlockStart"].get("start") is not None):
            print(f"{ block["contentBlockStart"].get("start") }", end="")



## Step 9: Demonstrate invoke_model_with_response_stream


Let's use invoke_model_with_streaming API, but without parsing.

In [ ]:
print("🚀 Starting invoke_model_with_response_stream, not parsed demonstration...")

try:
    # Make the API call
    response = client.invoke_model_with_response_stream(
        modelId=modelId,
        body=json.dumps(body)
    )
    
    # Get the response stream
    response_stream = response.get('body')
    
    # Process the stream using our decorated function
    print("\n\n----------------  un-parsed stream ---------------\n")
    for event in response_stream:
        print_helper(event)
    
    
except Exception as e:
    print(f"❌ Error during invoke_model_with_response_stream: {e}")
    print("   Make sure your AWS credentials are configured and you have access to Bedrock Nova models")

Now, let's use the first API method with our decorated function:

In [ ]:
print("🚀 Starting invoke_model_with_response_stream parsed demonstration...")

try:
    # Make the API call
    response = client.invoke_model_with_response_stream(
        modelId=modelId,
        body=json.dumps(body)
    )
    
    # Get the response stream
    response_stream = response.get('body')
    
    # print("📡 Response stream received, processing events...")
    # print("-" * 40)
    
    # Process the stream using our decorated function
    # This will extract content from <thinking> tags
    print("\n\n----------------  parsed stream ---------------\n") 
    for event in process_invoke_model_with_response_stream(response_stream):
        print_helper(event)
    
except Exception as e:
    print(f"❌ Error during invoke_model_with_response_stream: {e}")
    print("   Make sure your AWS credentials are configured and you have access to Bedrock Nova models")

## Step 10: Demonstrate converse_stream

Now let's use the second API method with our other decorated function:

Let's use the converse streaming API, but without parsing.

In [ ]:
print("🚀 Starting converse_stream un-parsed demonstration...")

try:
    # Make the API call using converse_stream
    response = client.converse_stream(
        modelId=LITE_MODEL_ID,
        messages=body["messages"],
        system=body["system"],
        inferenceConfig=body["inferenceConfig"],
        toolConfig=body["toolConfig"]
    )
    
    # Get the response stream
    response_stream = response.get('stream')
    
    
    # Process the stream using our decorated function
    # This will extract content from <thinking> tags
    print("\n\n----------------  un-parsed stream ---------------\n") 
    for event in response_stream:
        print_helper(event, "CONVERSE")
    
except Exception as e:
    print(f"❌ Error during converse_stream: {e}")
    print("   Make sure your AWS credentials are configured and you have access to Bedrock Nova models")

Now, let's use the second API method with our other decorated function:

In [ ]:
print("🚀 Starting converse_stream parsed demonstration...")

try:
    # Make the API call using converse_stream
    response = client.converse_stream(
        modelId=LITE_MODEL_ID,
        messages=body["messages"],
        system=body["system"],
        inferenceConfig=body["inferenceConfig"],
        toolConfig=body["toolConfig"]
    )
    
    # Get the response stream
    response_stream = response.get('stream')
    
    
    # Process the stream using our decorated function
    # This will extract content from <thinking> tags
    print("\n\n----------------  parsed stream ---------------\n") 
    for event in process_converse_stream(response_stream):
        print_helper(event, "CONVERSE")
    
except Exception as e:
    print(f"❌ Error during converse_stream: {e}")
    print("   Make sure your AWS credentials are configured and you have access to Bedrock Nova models")

## Step 11: Understanding the Output

Let's break down what the NovaStreamParser decorators do:

### How NovaStreamParser Works

1. **Decoration**: The decorators wrap your stream processing functions
2. **Stream Interception**: The original response stream is intercepted and processed
3. **Event Processing**: Different event types are handled appropriately:
   - `messageStart`: Initialization
   - `contentBlockDelta`: Content processing and tag extraction
   - `messageStop`: Finalization
   - `metadata`: Stream completion
4. **Tag Extraction**: Content within the specified XML tags (`<thinking>` in our case) is extracted
5. **Stream Generation**: A new processed stream is generated with the extracted content

### Key Benefits

- **Real-time Processing**: Processes streaming responses as they arrive
- **XML Tag Extraction**: Cleanly separates reasoning content from regular responses
- **Dual API Support**: Works with both streaming APIs
- **Event-based**: Handles different types of streaming events appropriately

## Step 12: Customization Examples

Here are some ways you can customize the NovaStreamParser for different use cases:

In [ ]:
# Example 1: Extract different XML tags
@parse_converse_stream("analysis")
def process_analysis_tags(response_stream):
    """Extract content from <analysis> tags"""
    return response_stream

# Example 2: Extract code blocks
@parse_invoke_model_with_response_stream(target_tag_name="code")
def process_code_blocks(response_stream):
    """Extract content from <code> tags"""
    return response_stream

# Example 3: Extract reasoning steps
@parse_converse_stream("reasoning")
def process_reasoning_steps(response_stream):
    """Extract content from <reasoning> tags"""
    return response_stream

print("✅ Custom extraction functions defined:")
print("   - process_analysis_tags: Extracts <analysis> content")
print("   - process_code_blocks: Extracts <code> content")
print("   - process_reasoning_steps: Extracts <reasoning> content")

## Step 13: Error Handling and Best Practices

Here are some best practices when using NovaStreamParser:

In [ ]:
# Best Practice 1: Always handle exceptions
def safe_stream_processing(client, model_id, body):
    """Safely process streams with proper error handling"""
    try:
        response = client.converse_stream(
            modelId=model_id,
            messages=body["messages"],
            system=body["system"],
            inferenceConfig=body["inferenceConfig"],
            toolConfig=body.get("toolConfig")  # Use .get() for optional parameters
        )
        
        response_stream = response.get('stream')
        
        for event in process_converse_stream(response_stream):
            yield event
            
    except Exception as e:
        print(f"Error processing stream: {e}")
        return None

# Best Practice 2: Validate inputs
def validate_configuration(body, model_id):
    """Validate configuration before making API calls"""
    required_keys = ["messages", "system", "inferenceConfig"]
    
    for key in required_keys:
        if key not in body:
            raise ValueError(f"Missing required key: {key}")
    
    if not model_id:
        raise ValueError("Model ID is required")
    
    if not body["messages"]:
        raise ValueError("At least one message is required")
    
    return True

print("✅ Best practices functions defined:")
print("   - safe_stream_processing: Handles exceptions gracefully")
print("   - validate_configuration: Validates inputs before API calls")

## Summary

You've successfully learned how to use NovaStreamParser! Here's what we covered:

### Key Concepts:
1. **Decorator Pattern**: Using `@parse_converse_stream` and `@parse_invoke_model_with_response_stream`
2. **XML Tag Extraction**: Extracting content from specified tags (like `<thinking>`)
3. **Dual API Support**: Working with both Bedrock streaming APIs
4. **Real-time Processing**: Handling streaming responses as they arrive

### Implementation Steps:
1. Import the NovaStreamParser decorators
2. Define decorated functions for stream processing
3. Configure system prompts to use XML tags
4. Set up inference and tool configurations
5. Create conversation messages
6. Initialize AWS Bedrock client
7. Process streams using decorated functions

### Next Steps:
- Experiment with different XML tag names
- Try different model configurations
- Implement custom processing logic within your decorated functions
- Add error handling and logging for production use

The NovaStreamParser makes it easy to extract structured content from Nova model responses while maintaining the benefits of streaming APIs!